In [1]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [2]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
# chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [3]:
url='https://pmiatlanta.org'

driver.get(url)
time.sleep(5)

soup=bs(driver.page_source,'lxml')
cards=soup.find('ul',{'class':'ebm-upcoming-events ebm-upcoming-events-improved'}).find_all('li',{'class':'row-fluid'})
len(cards)

items=[]
for i in cards:
    
    ritle=' '.join(i.find('a',{'class':'url ebm-event-link'}).text.split())
    link='https://pmiatlanta.org'+i.find('a')['href']
    da=' '.join(i.find('div',{'class':'ebm-event-date'}).text.split())
    items.append([ritle,link,da])


In [25]:
pmi=[]
for m in items:
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(3)
    link=m[1]
    
    soupn=bs(driver.page_source,'lxml')
    title=' '.join(soupn.find('h2',{'class':'pro-page-title-sub-heading'}).text.split())
    event_info=' '.join(soupn.find('div',{'class':'eb-description-details clearfix'}).find('p').text.split())
    hea=soupn.find('div',{'class':'eb-description-details clearfix'}).find_all('h3')
    for i in hea:
        if 'Date' in str(i):
            pinn=' '.join(i.find_next("p").text.split())
        elif 'When' in str(i):
            pinn=' '.join(i.find_next("p").text.split())
    if '(' in pinn:
        pinn=pinn.split('(')[0].strip()
            
    pinn=pinn.replace('–','-').replace('|','').replace('to','-').replace('TO','-').replace('c-ber','ctober').replace('C-BER','CTOBER').strip()
    for f in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']:
        if f in pinn:
            pinn=pinn.replace(f,'').strip()
    
    dat=re.search('[A-Za-z]{3,9}\s?\d+\W?\s?\d{4}',pinn).group()
    stt,ent=pinn.replace(dat,'').replace(',','').strip().split('-')
    snt,tre,z=[stt.strip().replace(' ',''),ent.strip().replace(' ',''),'']
    trabaye={
        'type':'general',
        'Start_time':snt,
        'end_time':tre,
        'timezone':z,
        'days':'all'
    }
    time_=json.dumps([trabaye],ensure_ascii=False)
    
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat
    if '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('and')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('&')[1].strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    else:
        st=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('Nd','').replace('St','').replace('Th','').replace('Rd','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
   
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v, '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v, '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
    tic=' '.join(soupn.find('td',{'class':'eb-event-property-value eb_price'}).text.split())
    if '$' not in tic:
        tic='$'+tic
        
    if tic!='':
        if 'Free' in tic or tic=='Free':
            ty='free'
            am=''
            cu=''

        elif re.search('[0-9]+\s?\$?\£?\€?\W?',tic).group():
            ty='paid'
            am=re.search('[0-9]+',tic).group()
            cu=re.search('\$?\£?\€?',tic).group()
        ticket=[{'type': ty,
                 'price':am,
                 'currency': cu}]
        ticket_list=json.dumps(ticket,ensure_ascii=False)
    else:
        ticket_list=''
        
    
    loc=' '.join(soupn.find_all('td',{'class':'eb-event-property-value'})[-1].text.split())
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            try:
                return(gu(location))
            except:
                sha=location.split(',')
                try:
                    gu(sha[-3])
                except:
                    try:
                        gu(sha[-2])
                    except:
                        try:
                            gu(sha[-1])
                        except Exception as e:
                            print(location, "; url didn't go through")
                            print(e)
                            return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    try:
                        google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
                    except:
                        google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
                        time.sleep(randint(0,3))
                        driver.get(google_url_for_location)
                        time.sleep(4)
                        google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")

    org_name='PMI Atlanta Chapter'
    org_web='https://pmiatlanta.org/'
    org_pro='At PMI Atlanta, we strive to have a significant impact on businesses in the metro Atlanta area, and we focus on increasing the recognition of the importance of project management. We offer quality programs, professional networking and high value education to anyone actively engaged or interested in project management principles and techniques.'
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    category=''
    Speakerlist='' 
#     ticket_list=''
    event_web=link
    mail_=json.dumps(['memberquestions@pmiatlanta.org'])
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc or 'ebinar' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        city=loc#.split()[-3]
        country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)
    
    pmi.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])


In [26]:
len(pmi)

13

In [27]:
pmi

[['https://pmiatlanta.org/events/event-list/2022802',
  'PMI Atlanta Chapter - PMI Atlanta Toastmasters (Virtual) Meeting - 220908',
  '2022-09-08',
  '2022-09-08',
  '[{"type": "general", "Start_time": "6:15PM", "end_time": "8:00PM", "timezone": "", "days": "all"}]',
  'If you would like to earn PDUs while improving your communications and leadership skills, this opportunity is for you! PMI Atlanta has teamed up with Toastmasters International to offer you a chance to hone your public speaking, critical listening, and leadership skills in a supportive environment. You will also have an opportunity to practice speaking off-the-cuff, giving and receiving feedback, emceeing a meeting, and many other valuable business skills. Since this club is exclusively for PMI Atlanta members, you will also have an opportunity to develop bonds within the PMI community.',
  '[{"type": "paid", "price": "6", "currency": "$"}]',
  'At PMI Atlanta, we strive to have a significant impact on businesses in th

In [28]:
pmi_dict=pmi


pmi_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=pmi_dict)
pmi_df.to_csv("pmi.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [ ]:
# driver.switch_to.window(driver.window_handles[0])

In [ ]:
# s=1
# for k in cards:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[0])
#     s+=1
# print(s+1)

In [7]:
# s=1
# for k in items:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[1])
#     s+=1
# print(s+1)

15


In [ ]:
# for k in range(1,31):
#     time.sleep(1)
#     driver.switch_to.window(driver.window_handles[k])
